In [ ]:
"""
Created on Wed Feb 01 15:27 2023

Try permute and predict with Smith data

Author: @claraburgard

"""

In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
from tqdm.notebook import trange, tqdm
import glob
import matplotlib as mpl
import seaborn as sns
import datetime
import time
import os,sys

import tensorflow as tf
from tensorflow import keras
from contextlib import redirect_stdout

from basal_melt_neural_networks.constants import *
import basal_melt_neural_networks.diagnostic_functions as diag
import basal_melt_neural_networks.data_formatting as dfmt
import basal_melt_neural_networks.postprocessing_functions as pp
from basal_melt_param.constants import *

In [ ]:
%matplotlib qt5

DEFINE OPTIONS

In [ ]:
mod_size =  'small' #'mini', 'small', 'medium', 'large', 'extra_large'
TS_opt = 'extrap' # extrap, whole, thermocline
norm_method =  'std' # std, interquart, minmax
exp_name = 'newbasic2'#'onlyTSdraftandslope' #'onlyTSdraftandslope' #'TSdraftbotandiceddandwcd' #'onlyTSisfdraft' #'TSdraftbotandiceddandwcdreldGL' #TSdraftslopereldGL

In [ ]:
#input_vars = ['dGL','dIF','corrected_isfdraft','bathy_metry','slope_bed_lon','slope_bed_lat','slope_ice_lon','slope_ice_lat','isfdraft_conc','theta_in','salinity_in','u_tide']
if exp_name == 'onlyTSdraft':
    input_vars = ['corrected_isfdraft','theta_in','salinity_in']
elif exp_name == 'TSdraftbotandiceddandwcd':
    input_vars = ['corrected_isfdraft','theta_in','salinity_in','water_col_depth','theta_bot','salinity_bot']
elif exp_name == 'TSdraftbotandiceddandwcdreldGL':
    input_vars = ['corrected_isfdraft','theta_in','salinity_in','water_col_depth','theta_bot','salinity_bot','rel_dGL']
elif exp_name == 'onlyTSdraftandslope':
    input_vars = ['corrected_isfdraft','theta_in','salinity_in','slope_ice_lon','slope_ice_lat']
elif exp_name == 'onlyTSdraft2':
    input_vars = ['corrected_isfdraft','theta_in','salinity_in']
elif exp_name == 'TSTfdGLdIFwcd':
    input_vars = ['corrected_isfdraft','theta_in','salinity_in','dGL','dIF','slope_ice_lon','slope_ice_lat','water_col_depth']
elif exp_name == 'TSdraftslopereldGL':
    input_vars = ['corrected_isfdraft','theta_in','salinity_in','slope_ice_lon','slope_ice_lat','rel_dGL']
elif exp_name == 'allbutconstants':
    input_vars = ['dGL','dIF','corrected_isfdraft','bathy_metry','slope_bed_lon','slope_bed_lat','slope_ice_lon','slope_ice_lat',
                'isfdraft_conc','theta_in','salinity_in','u_tide']
elif exp_name == 'newbasic':
    input_vars = ['dGL','dIF','corrected_isfdraft','bathy_metry','slope_bed_lon','slope_bed_lat','slope_ice_lon','slope_ice_lat',
                'theta_in','salinity_in']
elif exp_name == 'newbasic2':
    input_vars = ['dGL','dIF','corrected_isfdraft','bathy_metry','slope_bed_lon','slope_bed_lat','slope_ice_lon','slope_ice_lat',
                'theta_in','salinity_in','T_mean', 'S_mean', 'T_std', 'S_std']

In [ ]:
nemo_run = 'bf663'

In [ ]:

#isf_dim = [10,11,12,13,18,22,23,24,25,30,31,33,38,39,40,42,43,44,45,47,48,51,52,53,54,55,58,61,65,66,69,70,71,73,75]
#isf_dim = [10,11,12,13,18,22,23,25,30,31,33,39,40,42,43,44,45,47,51,55,58,61,65,66,69,70,71,73,75] # for bi646
#isf_dim = [10,11,12,13,18,22,23,25,30,31,33,39,40,42,43,44,45,47,51,55,58,61,65,66,69,70,71,73,75]

READ IN DATA

In [ ]:
inputpath_data_nn = '/bettik/burgardc/DATA/NN_PARAM/interim/INPUT_DATA/'


In [ ]:
if TS_opt == 'extrap':
    inputpath_CVinput = inputpath_data_nn+'EXTRAPOLATED_ISFDRAFT_CHUNKS/'
    inputpath_csv = inputpath_data_nn+'SMITH_'+nemo_run+'_EXTRAPDRAFT_CHUNKS/'
elif TS_opt == 'whole':
    inputpath_CVinput = inputpath_data_nn+'WHOLE_PROF_CHUNKS_CV/'
    path_orig_data = inputpath_data_nn+'WHOLE_PROF_CHUNKS/'
elif TS_opt == 'thermocline':
    inputpath_CVinput = inputpath_data_nn+'THERMOCLINE_CHUNKS_CV/'

In [ ]:
map_lim = [-3000000,3000000]
inputpath_data='/bettik/burgardc/DATA/NN_PARAM/interim/SMITH_'+nemo_run+'/'
file_other = xr.open_dataset(inputpath_data+'corrected_draft_bathy_isf.nc')#, chunks={'x': chunk_size, 'y': chunk_size})
file_other_cut = dfmt.cut_domain_stereo(file_other, map_lim, map_lim)
file_conc = xr.open_dataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc')
file_conc_cut = dfmt.cut_domain_stereo(file_conc, map_lim, map_lim)


In [ ]:
def read_input_evalmetrics_NN_yy(nemo_run, tt, file_conc, file_other):
    inputpath_boxes = '/bettik/burgardc/DATA/NN_PARAM/interim/BOXES/SMITH_'+nemo_run+'/'
    inputpath_data='/bettik/burgardc/DATA/NN_PARAM/interim/SMITH_'+nemo_run+'/'
    inputpath_mask='/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/SMITH_'+nemo_run+'/'
    
    file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(tt)+'.nc').drop('time')
    #file_isf = file_isf_orig.sel(Nisf=isf_list)
    nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    if 'labels' in file_isf.coords.keys():
        file_isf = file_isf.drop('labels')
    
    file_other_cut = file_other.sel(time=tt)
    file_conc_cut = file_conc.sel(time=tt)

    ice_draft_pos = file_other_cut['corrected_isfdraft']
    
    box_charac_2D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_2D_oneFRIS_'+str(tt)+'_merged75.nc')
    box_charac_1D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_1D_oneFRIS_'+str(tt)+'_merged75.nc')
    
    isf_stack_mask = dfmt.create_stacked_mask(file_isf['ISF_mask'], file_isf.Nisf, ['y','x'], 'mask_coord')

    file_isf_conc = file_conc_cut['isfdraft_conc']

    xx = file_isf.x
    yy = file_isf.y
    dx = (xx[2] - xx[1]).values
    dy = (yy[2] - yy[1]).values
    grid_cell_area = abs(dx*dy)  
    grid_cell_area_weighted = file_isf_conc * grid_cell_area
    
    geometry_info_2D = xr.merge([ice_draft_pos.rename('ice_draft_pos'),
                            grid_cell_area_weighted.rename('grid_cell_area_weighted'),
                            file_isf_conc])
    
    return file_isf, geometry_info_2D, box_charac_2D, box_charac_1D, isf_stack_mask

PREPARE FILES WITH EVERYTHING AND SHUFFLE THEN 

In [ ]:
startyy = 1980
endyy = 1980 + 60
n=0



for tt in tqdm(range(startyy,endyy)):
    file_isf, geometry_info_2D, box_charac_2D, box_charac_1D, isf_stack_mask = pp.read_input_evalmetrics_NN_yy(nemo_run, tt, file_conc_cut, file_other_cut)

    for kisf in file_isf.Nisf.values:
        df_nrun = pd.read_csv(inputpath_csv + 'dataframe_input_isf'+str(kisf).zfill(3)+'_'+str(tt).zfill(2)+'_'+nemo_run+'.csv')#.drop(['x', 'y'])
        if n == 0:
            print('here')
            df_list = df_nrun#.reset_index(drop=True, inplace=True)
        else:
            df_list = df_list.append(df_nrun) #.drop(['x', 'y'], axis=1, inplace=True))
        n = n+1

df_list.to_csv(inputpath_csv + 'dataframe_input_allisf_'+str(startyy)+'-'+str(endyy)+'_'+nemo_run+'.csv')
        

In [ ]:
startyy = 1980 
endyy = 1980 + 60
df_list_orig = pd.read_csv(inputpath_csv + 'dataframe_input_allisf_'+str(startyy)+'-'+str(endyy)+'_'+nemo_run+'.csv')
shuffled_df = df_list_orig.sample(frac=1, random_state=1)
shuffled_df.to_csv(inputpath_csv + 'dataframe_shuffledinput_allisf_'+str(startyy)+'-'+str(endyy)+'_'+nemo_run+'.csv')

COMBINE ALL DATA TO SHUFFLE (BOTH SMITH RUNS)

In [ ]:
startyy = 1980 
endyy = 1980 + 60

In [ ]:
data_bf663 = pd.read_csv(inputpath_data_nn+'SMITH_bf663_EXTRAPDRAFT_CHUNKS/dataframe_shuffledinput_allisf_'+str(startyy)+'-'+str(endyy)+'_bf663.csv')
data_bi646 = pd.read_csv(inputpath_data_nn+'SMITH_bi646_EXTRAPDRAFT_CHUNKS/dataframe_shuffledinput_allisf_'+str(startyy)+'-'+str(endyy)+'_bi646.csv')

In [ ]:
with open(inputpath_data_nn+'BOTH_SMITHS/dataframe_shuffledinput_allisf_'+str(startyy)+'-'+str(endyy)+'_bf663andbi646.csv','a') as f:
    for df in [data_bf663,data_bi646]:
        df.to_csv(f)
        f.write("\n")

In [ ]:
data_both = pd.read_csv(inputpath_data_nn+'BOTH_SMITHS/dataframe_shuffledinput_allisf_'+str(startyy)+'-'+str(endyy)+'_bf663andbi646.csv')

In [ ]:
data_both

APPLY 1D MODEL

In [ ]:
#continue with u_tide
outputpath_melt_nn = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/SMITH_'+nemo_run+'/'
path_model = '/bettik/burgardc/DATA/NN_PARAM/interim/NN_MODELS/experiments/WHOLE/'

startyy = 1980
endyy = 1980 + 30

df_shuffled = pd.read_csv(inputpath_csv + 'dataframe_shuffledinput_allisf_'+str(startyy)+'-'+str(endyy)+'_'+nemo_run+'.csv')

#for vv in ['dGL']: 
for vv in input_vars: 
    print(vv)
    
    res_1D_allyy_list = []
    for tt in tqdm(range(startyy,endyy + 1)):
    #for tt in tqdm([1970]):

        file_isf, geometry_info_2D, box_charac_2D, box_charac_1D, isf_stack_mask = read_input_evalmetrics_NN_yy(nemo_run, tt, file_conc_cut, file_other_cut)

        res_1D_list = []
        for kisf in file_isf.Nisf.values:  
        #for kisf in [66]:  


            norm_metrics_file = xr.open_dataset(inputpath_CVinput + 'metrics_norm_wholedataset.nc')
            #norm_metrics_file_addvar1 = xr.open_dataset(inputpath_CVinput + 'metrics_norm_addvar1_CV_noisf'+str(isf_out).zfill(3)+'_notblock'+str(tblock_out).zfill(3)+'.nc')
            #norm_metrics_file_addvar1 = norm_metrics_file_addvar1.drop('salinity_in')
            #norm_metrics_file = xr.merge([norm_metrics_file_orig,norm_metrics_file_addvar1])
            norm_metrics = norm_metrics_file.sel(norm_method=norm_method).drop('norm_method').to_dataframe()

            df_nrun = pd.read_csv(inputpath_csv + 'dataframe_input_isf'+str(kisf).zfill(3)+'_'+str(tt)+'_'+nemo_run+'.csv',index_col=[0,1])
            #df_nrun_addvar1 = pd.read_csv(path_orig_data + 'dataframe_addvar1_isf'+str(kisf).zfill(3)+'_'+str(tblock_out).zfill(3)+'.csv',index_col=[0,1,2])
            #df_nrun_addvar1 = df_nrun_addvar1.drop(['salinity_in'], axis=1)
            #df_nrun = pd.concat([df_nrun_orig,df_nrun_addvar1],join = 'outer', axis = 1)
            
            nrows = len(df_nrun.index)
            shuffled_var = df_shuffled[vv].sample(n=nrows).values
            df_nrun_in_shuffled = df_nrun.drop(vv, axis=1).copy()
            df_nrun_in_shuffled[vv] = shuffled_var
            
            ens_res2D_list = []
            for seed_nb in range(1,11):
                model = keras.models.load_model(path_model + 'model_nn_'+mod_size+'_'+exp_name+'_wholedataset_'+str(seed_nb).zfill(2)+'_TS'+TS_opt+'_norm'+norm_method+'.h5')

                res_2D = pp.apply_NN_results_2D_1isf_1tblock(file_isf, norm_metrics, df_nrun_in_shuffled, model, input_vars)

                ens_res2D_list.append(res_2D.assign_coords({'seed_nb': seed_nb}))

            xr_ens_res2D = xr.concat(ens_res2D_list, dim='seed_nb')
            xr_ensmean_res2D = xr_ens_res2D.mean('seed_nb')
            
            if box_charac_2D and box_charac_1D:
                box_loc_config2 = box_charac_2D['box_location'].sel(box_nb_tot=box_charac_1D['nD_config'].sel(config=2))
                box1 = box_loc_config2.where(box_loc_config2==1).isel(Nisf=0).drop('Nisf')

            geometry_isf_2D = dfmt.choose_isf(geometry_info_2D,isf_stack_mask, kisf)
            melt_rate_2D_isf_m_per_y = dfmt.choose_isf(xr_ensmean_res2D,isf_stack_mask, kisf)

            melt_rate_1D_isf_Gt_per_y = (melt_rate_2D_isf_m_per_y * geometry_isf_2D['grid_cell_area_weighted']).sum(dim=['mask_coord']) * rho_i / 10**12

            box_loc_config_stacked = dfmt.choose_isf(box1, isf_stack_mask, kisf)
            param_melt_2D_box1_isf = melt_rate_2D_isf_m_per_y.where(np.isfinite(box_loc_config_stacked))

            melt_rate_1D_isf_myr_box1_mean = dfmt.weighted_mean(param_melt_2D_box1_isf,['mask_coord'], geometry_isf_2D['isfdraft_conc'])     

            out_1D = xr.concat([melt_rate_1D_isf_Gt_per_y, melt_rate_1D_isf_myr_box1_mean], dim='metrics').assign_coords({'metrics': ['Gt','box1']})

            res_1D_list.append(out_1D)

        res_1D_all = xr.concat(res_1D_list, dim='Nisf')
        res_1D_allyy_list.append(res_1D_all)

    res_1D_allyy = xr.concat(res_1D_allyy_list, dim='time')

    res_1D_allyy.to_netcdf(outputpath_melt_nn + 'evalmetrics_shuffled'+vv+'_1D_'+mod_size+'_'+exp_name+'_ensmean_'+TS_opt+'_norm'+norm_method+'_allyy_'+nemo_run+'.nc')



GROUP SOME OF THE SHUFFLING

In [ ]:
#continue with u_tide
outputpath_melt_nn = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/SMITH_'+nemo_run+'/'
path_model = '/bettik/burgardc/DATA/NN_PARAM/interim/NN_MODELS/experiments/WHOLE/'

if nemo_run == 'bf663':
    df_shuffled = pd.read_csv(inputpath_csv + 'dataframe_shuffledinput_allisf_timeuntil40_'+nemo_run+'.csv')
elif nemo_run == 'bi646':
    df_shuffled = pd.read_csv(inputpath_csv + 'dataframe_shuffledinput_allisf_timeuntil35_'+nemo_run+'.csv')

groups_list = ['position','watercolumn','bathy_geometry','ice_geometry','water_properties']

for groupvv in groups_list:
    
    print(groupvv)
    if groupvv == 'position':
        var_of_int = ['dGL','dIF']
    elif groupvv == 'watercolumn':
        var_of_int = ['corrected_isfdraft','bathy_metry']
    elif groupvv == 'bathy_geometry':
        var_of_int = ['bathy_metry','slope_bed_lon','slope_bed_lat']
    elif groupvv == 'ice_geometry':
        var_of_int = ['corrected_isfdraft','slope_ice_lon','slope_ice_lat']
    elif groupvv == 'water_properties':
        var_of_int = ['theta_in','salinity_in']
    
    res_1D_allyy_list = []
    for tt in tqdm(tblock_dim):
    #for tt in tqdm([1970]):
        isf_out = 0
        tblock_out = 5

        file_isf, geometry_info_2D, box_charac_2D, box_charac_1D, isf_stack_mask = read_input_evalmetrics_NN_yy(nemo_run, tt, isf_dim)

        res_1D_list = []
        for kisf in isf_dim:  
        #for kisf in [66]:  


            norm_metrics_file = xr.open_dataset(inputpath_CVinput + 'metrics_norm_wholedataset.nc')
            #norm_metrics_file_addvar1 = xr.open_dataset(inputpath_CVinput + 'metrics_norm_addvar1_CV_noisf'+str(isf_out).zfill(3)+'_notblock'+str(tblock_out).zfill(3)+'.nc')
            #norm_metrics_file_addvar1 = norm_metrics_file_addvar1.drop('salinity_in')
            #norm_metrics_file = xr.merge([norm_metrics_file_orig,norm_metrics_file_addvar1])
            norm_metrics = norm_metrics_file.sel(norm_method=norm_method).drop('norm_method').to_dataframe()

            df_nrun = pd.read_csv(inputpath_csv + 'dataframe_input_isf'+str(kisf).zfill(3)+'_'+str(tt)+'_'+nemo_run+'.csv',index_col=[0,1])
            #df_nrun_addvar1 = pd.read_csv(path_orig_data + 'dataframe_addvar1_isf'+str(kisf).zfill(3)+'_'+str(tblock_out).zfill(3)+'.csv',index_col=[0,1,2])
            #df_nrun_addvar1 = df_nrun_addvar1.drop(['salinity_in'], axis=1)
            #df_nrun = pd.concat([df_nrun_orig,df_nrun_addvar1],join = 'outer', axis = 1)

            model = keras.models.load_model(path_model + 'model_nn_'+mod_size+'_'+exp_name+'_wholedataset_TS'+TS_opt+'_norm'+norm_method+'.h5')
            
            nrows = len(df_nrun.index)
            shuffled_var = df_shuffled[var_of_int].sample(n=nrows).values
            df_nrun_in_shuffled = df_nrun.drop(var_of_int, axis=1).copy()
            df_nrun_in_shuffled[var_of_int] = shuffled_var

            res_1D = pp.evalmetrics_1D_NN(kisf, norm_metrics, df_nrun_in_shuffled, model, file_isf, geometry_info_2D, box_charac_2D, box_charac_1D, isf_stack_mask, input_vars)    
            res_1D_list.append(res_1D)

        res_1D_all = xr.concat(res_1D_list, dim='Nisf')
        res_1D_allyy_list.append(res_1D_all)

    res_1D_allyy = xr.concat(res_1D_allyy_list, dim='time')

    res_1D_allyy.to_netcdf(outputpath_melt_nn + 'evalmetrics_shuffled'+groupvv+'_1D_'+mod_size+'_'+exp_name+'_'+TS_opt+'_norm'+norm_method+'_allyy_'+nemo_run+'.nc')




APPLY MODEL

In [ ]:
### use any model from CV over time
outputpath_melt_nn = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/SMITH_'+nemo_run+'/'
path_model = '/bettik/burgardc/DATA/NN_PARAM/interim/NN_MODELS/experiments/WHOLE/'

#for tt in tqdm(tblock_dim):
for tt in tqdm([1970]):
    
    file_isf, geometry_info_2D, box_charac_2D, box_charac_1D, isf_stack_mask = read_input_evalmetrics_NN_yy(nemo_run, tt, isf_dim)
    
    res_2D_shuffled = []
    for vv in ['dGL','dIF']: #, 'dIF','corrected_isfdraft', 'bathy_metry', 'slope_bed_lon',
               #'slope_bed_lat', 'slope_ice_lon', 'slope_ice_lat',
               #'entry_depth_max', 'isfdraft_conc', 'u_tide', 'theta_in', 'salinity_in']:
        print(vv)
        
        res_2D_all = None
        #for kisf in isf_dim: 
        for kisf in [66]:
            
            norm_metrics_file = xr.open_dataset(inputpath_CVinput + 'metrics_norm_wholedataset.nc')
            #norm_metrics_file_addvar1 = xr.open_dataset(inputpath_CVinput + 'metrics_norm_addvar1_CV_noisf'+str(isf_out).zfill(3)+'_notblock'+str(tblock_out).zfill(3)+'.nc')
            #norm_metrics_file_addvar1 = norm_metrics_file_addvar1.drop('salinity_in')
            #norm_metrics_file = xr.merge([norm_metrics_file_orig,norm_metrics_file_addvar1])
            norm_metrics = norm_metrics_file.sel(norm_method=norm_method).drop('norm_method').to_dataframe()

            df_nrun = pd.read_csv(inputpath_csv + 'dataframe_input_isf'+str(kisf).zfill(3)+'_'+str(tt).zfill(2)+'_'+nemo_run+'.csv',index_col=[0,1])
            #df_nrun_addvar1 = pd.read_csv(path_orig_data + 'dataframe_addvar1_isf'+str(kisf).zfill(3)+'_'+str(tblock_out).zfill(3)+'.csv',index_col=[0,1,2])
            #df_nrun_addvar1 = df_nrun_addvar1.drop(['salinity_in'], axis=1)
            #df_nrun = pd.concat([df_nrun_orig,df_nrun_addvar1],join = 'outer', axis = 1)

            model = keras.models.load_model(path_model + 'model_nn_'+mod_size+'_'+exp_name+'_wholedataset_TS'+TS_opt+'_norm'+norm_method+'.h5')
            
            shuffled_var = df_nrun[vv].copy()
            np.random.shuffle(shuffled_var.values)
            df_nrun_in_shuffled = df_nrun.drop(vv, axis=1).copy()
            df_nrun_in_shuffled[vv] = shuffled_var
            #res_2D_shuffled = pp.apply_NN_results_2D_1isf_1tblock(file_isf, norm_metrics, df_nrun_in_shuffled, model, input_vars)
            res_2D = pp.apply_NN_results_2D_1isf_1tblock(file_isf, norm_metrics, df_nrun_in_shuffled, model, input_vars)

            if res_2D_all is None:
                res_2D_all = res_2D
            else:
                res_2D_all = res_2D_all.combine_first(res_2D)
        res_2D_shuffled.append(res_2D_all.assign_coords({'shuffle_dim': vv}))
        
    res_2D_shuffled_all = xr.concat(res_2D_shuffled, dim='shuffle_dim')        
    #res_2D_all.to_netcdf(outputpath_melt_nn + 'evalmetrics_2D_'+mod_size+'_'+exp_name+'_'+TS_opt+'_norm'+norm_method+'_'+str(tt)+'_'+nemo_run+'_shuffled.nc')


In [ ]:
np.sqrt((((res_1D_allyy['predicted_melt'] - res_1D_allyy['reference_melt']).sel(metrics='Gt'))**2).mean())